In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv('data/telco_customer_churn.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [3]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = ['gender', 
               'seniorcitizen', 
               'partner', 
               'dependents', 
               'phoneservice', 
               'multiplelines', 
               'internetservice', 
               'onlinesecurity', 
               'onlinebackup', 
               'deviceprotection', 
               'techsupport', 
               'streamingtv', 
               'streamingmovies', 
               'contract', 
               'paperlessbilling', 
               'paymentmethod']

In [4]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [5]:
def train(df, y, C=1.0):
    dicts = df[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y)

    return dv, model

In [6]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)

    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [7]:
C = 1
n_splits = 5

In [8]:
# turn off warning
import warnings
warnings.filterwarnings('ignore')

In [9]:
scores = []

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)
    
print('%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

1 0.842 +- 0.007


In [10]:
scores

[0.8449694248416294,
 0.8451343365117203,
 0.8334216335540838,
 0.8348888053974547,
 0.8516284396473284]

In [ ]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

Save the model

In [ ]:
import pickle

In [ ]:
output_file = 'model_C=%s.bin' % C
output_file

In [ ]:
# First way
f_out = open(output_file, 'wb')
pickle.dump((dv, model), f_out)
f_out.close()

In [ ]:
# Second way
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)
    # Do stuff

# Do others

Load the model

In [3]:
import pickle

In [5]:
input_file = 'model_C=1.bin'

with open(input_file, 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [7]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(C=1, max_iter=1000))

In [31]:
# For testing
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [33]:
X = dv.transform([customer])

In [35]:
model.predict_proba(X)[0, 1]

0.6271616080216115

In [38]:
jupyter nbconvert --to script "5.Deploying Model.ipynb"

SyntaxError: invalid syntax (3001149982.py, line 1)